# ALENN - Replication Notebook 
## Franke and Westerhoff (2012) Model, KDE

Donovan Platt
<br>
Mathematical Institute, University of Oxford
<br>
Institute for New Economic Thinking at the Oxford Martin School
<br>
<br>
Copyright (c) 2020, University of Oxford. All rights reserved.
<br>
Distributed under a BSD 3-Clause licence. See the accompanying LICENCE file for further details.

# 1. Modules and Packages
Load all required modules and packages.

In [1]:
# Import the ALENN ABM Estimation Package
import alenn

# Import Numerical Computation Libraries
import numpy as np
import pandas as pd

# Import General Mathematical Libraries
from scipy import stats

Using TensorFlow backend.


# 2. Estimation Experiments
Replication of the KDE experiments. Note that here we generate only a single Markov Chain as opposed to the 5 considered in the original paper.

## 2.1. Free Parameter Set HPM

### Model Specification

In [2]:
# Specify the Simulated Data Characteristics
T_emp = 1000    # Pseudo-empirical series length 
T_sim = 1000    # Length of each Monte Carlo replication
n = 100         # Number of Monte Carlo replications

# Specify the Pseudo-Empirical Data
empirical = np.diff(alenn.models.fw_hpm(0.01, 1, 0.12, 1.5, -0.327, 1.79, 18.43, 0.758, 2.087, 0, T_emp, 1, 1), axis = 0)[:, 0]

# Define the Candidate Model Function
def model(theta):
    return np.diff(alenn.models.fw_hpm(0.01, 1, 0.12, 1.5, theta[0], theta[1], theta[2], 0.758, theta[3], 0, T_sim, n, 7), axis = 0)

# Define Parameter Priors
priors = [stats.uniform(loc = -1, scale = 2).pdf,
          stats.uniform(loc = 0, scale = 2).pdf,
          stats.uniform(loc = 0, scale = 20).pdf,
          stats.uniform(loc = 0, scale = 5).pdf]

# Define the Parameter Bounds
theta_lower = np.array([-1, 0, 0, 0])
theta_upper = np.array([1, 2, 20, 5])

### Posterior Specification

In [3]:
# Create a KDE Posterior Approximator Object (Uses Default Settings from the Paper)
posterior = alenn.kde.KDEPosterior()

# Add the Model, Priors, and Empirical Data to the Newly-created Object
posterior.set_model(model)
posterior.set_prior(priors)
posterior.load_data(empirical)

------------------------------------------
Successfully created a new KDE object:
------------------------------------------

Using a Silverman bandwidth approximation.
------------------------------------------

Model function successfully set.
----------------------------------------------------------------------------

Model prior successfully set. The model has 4 free parameters.
----------------------------------------------------------------------------

Empirical data successfully loaded. There are 1000 observations in total.
----------------------------------------------------------------------------



### Sampler Specification

In [4]:
# Create an Adaptive MCMC Sampler Object
sampler = alenn.mcmc.AdaptiveMCMC(K = 70, S = 5000)

# Add the Posterior Approximator and Parameter Ranges to the Newly-created Object
sampler.set_posterior(posterior)
sampler.set_initialisation_ranges(theta_lower, theta_upper)

# Initiate the Sampling Process
sampler.sample_posterior()

-----------------------------------------------
Successfully created a new MCMC sampler object:
-----------------------------------------------
Number of sample sets:         5000                
Number of samples per set:     70                  
-----------------------------------------------

KDEPosterior object successfully loaded.
----------------------------------------------------------------------------

Initialisation ranges successfully set.

           Lower Bound  Upper Bound
Parameter                          
1                   -1            1
2                    0            2
3                    0           20
4                    0            5
----------------------------------------------------------------------------



### Result Processing

In [5]:
# Process the Sampler Output
samples = sampler.process_samples(burn_in = 1500)

# Calculate the Posterior Mean
pos_mean = samples[:, :posterior.num_param].mean(axis = 0)

# Calculate the Posterior Standard Deviation
pos_std = samples[:, :posterior.num_param].std(axis = 0)

# Construct a Result Table
result_table = pd.DataFrame(np.array([pos_mean, pos_std]).transpose(), columns = ['Posterior Mean', 'Posterior Std. Dev.'])
result_table.index.name = 'Parameter'
result_table.index += 1

# Display the Result Table
print('Final Estimation Results:')
print('')
print(result_table)

Final Estimation Results:

           Posterior Mean  Posterior Std. Dev.
Parameter                                     
1               -0.130627             0.086259
2                1.870010             0.115560
3               16.224146             3.110913
4                2.295653             0.406562


## 2.2. Free Parameter Set WP

### Model Specification

In [5]:
# Specify the Simulated Data Characteristics
T_emp = 1000    # Pseudo-empirical series length 
T_sim = 1000    # Length of each Monte Carlo replication
n = 100         # Number of Monte Carlo replications

# Specify the Pseudo-Empirical Data
empirical = np.diff(alenn.models.fw_wp(0.01, 1, 1, 0.9, 2.1, 2668, 0.987, 0.752, 1.726, 0, T_emp, 1, 1), axis = 0)[:, 0]

# Define the Candidate Model Function
def model(theta):
    return np.diff(alenn.models.fw_wp(0.01, 1, 1, 0.9, 2.1, theta[0], theta[1], 0.752, theta[2], 0, T_sim, n, 7), axis = 0)

# Define Parameter Priors
priors = [stats.uniform(loc = 0, scale = 15000).pdf,
          stats.uniform(loc = 0, scale = 1).pdf,
          stats.uniform(loc = 0, scale = 5).pdf]

# Define the Parameter Bounds
theta_lower = np.array([0, 0, 0])
theta_upper = np.array([15000, 1, 5])

### Posterior Specification

In [6]:
# Create a KDE Posterior Approximator Object (Uses Default Settings from the Paper)
posterior = alenn.kde.KDEPosterior()

# Add the Model, Priors, and Empirical Data to the Newly-created Object
posterior.set_model(model)
posterior.set_prior(priors)
posterior.load_data(empirical)

------------------------------------------
Successfully created a new KDE object:
------------------------------------------

Using a Silverman bandwidth approximation.
------------------------------------------

Model function successfully set.
----------------------------------------------------------------------------

Model prior successfully set. The model has 3 free parameters.
----------------------------------------------------------------------------

Empirical data successfully loaded. There are 1000 observations in total.
----------------------------------------------------------------------------



### Sampler Specification

In [7]:
# Create an Adaptive MCMC Sampler Object
sampler = alenn.mcmc.AdaptiveMCMC(K = 70, S = 5000)

# Add the Posterior Approximator and Parameter Ranges to the Newly-created Object
sampler.set_posterior(posterior)
sampler.set_initialisation_ranges(theta_lower, theta_upper)

# Initiate the Sampling Process
sampler.sample_posterior()

-----------------------------------------------
Successfully created a new MCMC sampler object:
-----------------------------------------------
Number of sample sets:         5000                
Number of samples per set:     70                  
-----------------------------------------------

KDEPosterior object successfully loaded.
----------------------------------------------------------------------------

Initialisation ranges successfully set.

           Lower Bound  Upper Bound
Parameter                          
1                    0        15000
2                    0            1
3                    0            5
----------------------------------------------------------------------------



### Result Processing

In [5]:
# Process the Sampler Output
samples = sampler.process_samples(burn_in = 1500)

# Calculate the Posterior Mean
pos_mean = samples[:, :posterior.num_param].mean(axis = 0)

# Calculate the Posterior Standard Deviation
pos_std = samples[:, :posterior.num_param].std(axis = 0)

# Construct a Result Table
result_table = pd.DataFrame(np.array([pos_mean, pos_std]).transpose(), columns = ['Posterior Mean', 'Posterior Std. Dev.'])
result_table.index.name = 'Parameter'
result_table.index += 1

# Display the Result Table
print('Final Estimation Results:')
print('')
print(result_table)

Final Estimation Results:

           Posterior Mean  Posterior Std. Dev.
Parameter                                     
1             2286.509994          2596.313394
2                0.610277             0.280591
3                1.389007             0.136704
